In [1]:
import numpy as np
import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [3]:
stopwords = stopwords.words("english")

In [4]:
print (stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [5]:
data = load_data("vol7.json")["descriptions"]

In [6]:
!python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.1 MB/s eta 0:00:0031m16.3 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

# lemmatized version of the sentences

member shoot severely injure member Police open fire gathering supporter house follow disp


In [8]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

# text broken into individual words

['member', 'shoot', 'severely', 'injure', 'member', 'police', 'open', 'fire', 'gathering', 'supporter', 'house', 'follow', 'dispute', 'neighbour', 'link', 'other', 'councillor']


In [9]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

#this is the index of word in the created dictionary, its frequency

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]
councillor


In [18]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [19]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.519379  0.123313       1        1  16.401079
23     0.162925 -0.505936       2        1  13.919611
28     0.513472  0.153816       3        1   8.472936
1      0.239703  0.234410       4        1   5.800424
29     0.478303 -0.132784       5        1   5.037070
18     0.010929 -0.457687       6        1   3.968482
22    -0.360323  0.362646       7        1   3.962644
2      0.455618  0.034674       8        1   3.507395
7     -0.470451 -0.193996       9        1   3.278448
21    -0.473265 -0.031266      10        1   2.992536
11    -0.167967  0.257806      11        1   2.890103
24    -0.136872 -0.490840      12        1   2.645048
4     -0.339018 -0.385249      13        1   2.420094
13     0.279003 -0.022189      14        1   2.407277
20     0.050263  0.497497      15        1   2.320008
10     0.412345 -0.262638      16        1   2.194240
0      0.220734 -0.264761      17        1   2.125537
19    -0.390215  0.200037      18        1   2.030385
25    -0.332131 -0.227873      19        1   2.026984
16     0.120431  0.373963      20        1   2.009970
15     0.266212  0.415870      21        1   1.912006
14    -0.135818  0.383401      22        1   1.848117
12    -0.161653  0.486384      23        1   1.844397
8      0.281964 -0.391178      24        1   1.841534
9      0.376866  0.276461      25        1   1.664247
26    -0.132449 -0.279082      26        1   0.386540
6     -0.061481 -0.039476      27        1   0.023222
5     -0.062772 -0.038365      28        1   0.023222
17    -0.062384 -0.038539      29        1   0.023222
27    -0.062590 -0.038421      30        1   0.023222, topic_info=         Term          Freq         Total Category  logprob  loglift
15  supporter  23374.000000  23374.000000  Default  30.0000  30.0000
8      member  14422.000000  14422.000000  Default  29.0000  29.0000
6      injure   8197.000000   8197.000000  Default  28.0000  28.0000
14      shoot  11490.000000  11490.000000  Default  27.0000  27.0000
5       house   5783.000000   5783.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
25    several      0.015823    747.742887  Topic30  -8.2496  -2.3955
26       time      0.015823    955.399128  Topic30  -8.2496  -2.6406
27      baton      0.015823     24.966826  Topic30  -8.2496   1.0040
28       beat      0.015823   2605.786940  Topic30  -8.2496  -3.6439
29       butt      0.015823     57.011809  Topic30  -8.2496   0.1783

[1483 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
323      17  0.989627    about
259       3  0.994494       ac
251      18  0.997558   accuse
168      17  0.991567   acquit
308      12  0.996496      act
...     ...       ...      ...
295      10  0.997136     work
118      16  0.997831    wound
258       6  0.982082  wrongly
228      14  0.999053     year
95       24  0.998326    youth

[330 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 24, 29, 2, 30, 19, 23, 3, 8, 22, 12, 25, 5, 14, 21, 11, 1, 20, 26, 17, 16, 15, 13, 9, 10, 27, 7, 6, 18, 28])